In [37]:
# importation des bibliothèques 
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt 
from statsmodels.stats.stattools import jarque_bera 
from statsmodels.tsa.stattools import adfuller 
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA 
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters() 
from pandas.tseries.offsets import MonthEnd
import seaborn as sns
from scipy import stats 
plt.style.use('ggplot')
import plotly.express as px
# la bibliothèque statsmodels offre un ensemble de fonctions permettant de travailler avec des données de séries temporelles 
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import *
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.stattools import acf, pacf
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
from pandas.tseries.offsets import MonthEnd
from datetime import datetime
# importation des librairies pour l'application WEB
from flask import Flask, flash, render_template, request, session, abort


# importation des données 
df = pd.read_csv("DataSmartCity.csv", sep = ',')



In [ ]:
df.shape # donne le nombre de ligne et colonne de la base 

(5256, 21)

In [ ]:
df.info() # connaitre le nom des colonnes et le type des tables 

In [ ]:
(df.isna().sum(axis=0)/df.shape[0]*100) #taux de donnees manquantes pour chaque colonne

In [ ]:
df.describe().T # description de toutes les colonnes 

In [ ]:
df.isnull().sum() # connaitre le nombre de valeurs manquantes dans chaques colonnes 

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [2]:
df.dropna(subset=['elec_energy_prod_hydroelectric_percent'], how='any', inplace=True)
df.dropna(subset=['elect_access_percent_population'], how='any', inplace=True)
df.dropna(subset=['population_largest_city'], how='any', inplace=True)
#df.dropna(subset=['elect_access_percent_population'], how='any', inplace=True)

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
df.isnull().sum() # connaitre le nombre de valeurs manquantes dans chaques colonnes 

In [ ]:
# Gestion des valeurs manquantes: On remplace les valeurs manquantes par la mediane 

countries = df["CountryName"].unique()
# Partie energie electrique 

cols_to_clean = list(df.columns)
cols_to_clean = cols_to_clean[4:]

for col_name in cols_to_clean:
  df[col_name + "_median_value"] = df.groupby(["CountryName"])[col_name].transform("median")
  df[col_name].fillna(df[col_name + "_median_value"], inplace=True)
df

In [ ]:
df.isnull().sum() # connaitre le nombre de valeurs manquantes dans chaques colonnes 

In [4]:
df.columns

Index(['Unnamed: 0', 'CountryCode', 'CountryName', 'Year',
       'elec_energy_prod_hydroelectric_percent',
       'elec_energy_prod_naturalgas_percent',
       'elec_energy_prod_nuclear_percent',
       'elec_energy_prod_oil_gas_coal_percent',
       'elec_energy_prod_renewable_percent', 'elect_access_percent_population',
       'elect_consumption_Kwh_per_capita', 'female_population_percent',
       'gdp_growth_annual_percent', 'population_0_14_percent',
       'population_15_64_percent', 'population_65_plus_percent',
       'population_density', 'population_largest_city',
       'rural_population_percent', 'surface_area', 'total_population',
       'elec_energy_prod_hydroelectric_percent_median_value',
       'elec_energy_prod_naturalgas_percent_median_value',
       'elec_energy_prod_nuclear_percent_median_value',
       'elec_energy_prod_oil_gas_coal_percent_median_value',
       'elec_energy_prod_renewable_percent_median_value',
       'elect_access_percent_population_median_value

On remarque que le coefficient de correlation entre population_15_64_percent et elect_access_percent_population est le plus elevé (0.701492)

In [ ]:
tw = pd.read_csv("twitter_2022.csv", sep = ';')
tw.head(5)

In [39]:
tw["start_date"] = pd.to_datetime(tw["start_date"]) 
tw

,Unnamed: 0,start_date,end_date,counts,keywords
0,0,2022-05-01 20:41:34+00:00,2022-05-01T21:00:00.000Z,85,electric cars
1,1,2022-05-01 21:00:00+00:00,2022-05-01T22:00:00.000Z,250,electric cars
2,2,2022-05-01 22:00:00+00:00,2022-05-01T23:00:00.000Z,199,electric cars
3,3,2022-05-01 23:00:00+00:00,2022-05-02T00:00:00.000Z,166,electric cars
4,4,2022-05-02 00:00:00+00:00,2022-05-02T01:00:00.000Z,154,electric cars
...,...,...,...,...,...
1685,1685,2022-05-08 16:00:00+00:00,2022-05-08T17:00:00.000Z,257,solar energy
1686,1686,2022-05-08 17:00:00+00:00,2022-05-08T18:00:00.000Z,367,solar energy
1687,1687,2022-05-08 18:00:00+00:00,2022-05-08T19:00:00.000Z,539,solar energy
1688,1688,2022-05-08 19:00:00+00:00,2022-05-08T20:00:00.000Z,378,solar energy


In [40]:
tw["end_date"] = pd.to_datetime(tw["end_date"]) 
tw

,Unnamed: 0,start_date,end_date,counts,keywords
0,0,2022-05-01 20:41:34+00:00,2022-05-01 21:00:00+00:00,85,electric cars
1,1,2022-05-01 21:00:00+00:00,2022-05-01 22:00:00+00:00,250,electric cars
2,2,2022-05-01 22:00:00+00:00,2022-05-01 23:00:00+00:00,199,electric cars
3,3,2022-05-01 23:00:00+00:00,2022-05-02 00:00:00+00:00,166,electric cars
4,4,2022-05-02 00:00:00+00:00,2022-05-02 01:00:00+00:00,154,electric cars
...,...,...,...,...,...
1685,1685,2022-05-08 16:00:00+00:00,2022-05-08 17:00:00+00:00,257,solar energy
1686,1686,2022-05-08 17:00:00+00:00,2022-05-08 18:00:00+00:00,367,solar energy
1687,1687,2022-05-08 18:00:00+00:00,2022-05-08 19:00:00+00:00,539,solar energy
1688,1688,2022-05-08 19:00:00+00:00,2022-05-08 20:00:00+00:00,378,solar energy


In [ ]:
tw.isnull().sum() # connaitre le nombre de valeurs manquantes dans chaques colonnes 

Unnamed: 0    0
start_date    0
end_date      0
counts        0
keywords      0
dtype: int64

In [41]:
tw['date'] = [d.date() for d in tw['start_date']]
tw
#tw['time'] = [d.time() for d in tw['start_date','end_date']]

,Unnamed: 0,start_date,end_date,counts,keywords,date
0,0,2022-05-01 20:41:34+00:00,2022-05-01 21:00:00+00:00,85,electric cars,2022-05-01
1,1,2022-05-01 21:00:00+00:00,2022-05-01 22:00:00+00:00,250,electric cars,2022-05-01
2,2,2022-05-01 22:00:00+00:00,2022-05-01 23:00:00+00:00,199,electric cars,2022-05-01
3,3,2022-05-01 23:00:00+00:00,2022-05-02 00:00:00+00:00,166,electric cars,2022-05-01
4,4,2022-05-02 00:00:00+00:00,2022-05-02 01:00:00+00:00,154,electric cars,2022-05-02
...,...,...,...,...,...,...
1685,1685,2022-05-08 16:00:00+00:00,2022-05-08 17:00:00+00:00,257,solar energy,2022-05-08
1686,1686,2022-05-08 17:00:00+00:00,2022-05-08 18:00:00+00:00,367,solar energy,2022-05-08
1687,1687,2022-05-08 18:00:00+00:00,2022-05-08 19:00:00+00:00,539,solar energy,2022-05-08
1688,1688,2022-05-08 19:00:00+00:00,2022-05-08 20:00:00+00:00,378,solar energy,2022-05-08


In [42]:
tw['time'] = [d.time() for d in tw['start_date']]
tw

,Unnamed: 0,start_date,end_date,counts,keywords,date,time
0,0,2022-05-01 20:41:34+00:00,2022-05-01 21:00:00+00:00,85,electric cars,2022-05-01,20:41:34
1,1,2022-05-01 21:00:00+00:00,2022-05-01 22:00:00+00:00,250,electric cars,2022-05-01,21:00:00
2,2,2022-05-01 22:00:00+00:00,2022-05-01 23:00:00+00:00,199,electric cars,2022-05-01,22:00:00
3,3,2022-05-01 23:00:00+00:00,2022-05-02 00:00:00+00:00,166,electric cars,2022-05-01,23:00:00
4,4,2022-05-02 00:00:00+00:00,2022-05-02 01:00:00+00:00,154,electric cars,2022-05-02,00:00:00
...,...,...,...,...,...,...,...
1685,1685,2022-05-08 16:00:00+00:00,2022-05-08 17:00:00+00:00,257,solar energy,2022-05-08,16:00:00
1686,1686,2022-05-08 17:00:00+00:00,2022-05-08 18:00:00+00:00,367,solar energy,2022-05-08,17:00:00
1687,1687,2022-05-08 18:00:00+00:00,2022-05-08 19:00:00+00:00,539,solar energy,2022-05-08,18:00:00
1688,1688,2022-05-08 19:00:00+00:00,2022-05-08 20:00:00+00:00,378,solar energy,2022-05-08,19:00:00


In [ ]:
tw['time1'] = [d.time() for d in tw['end_date']]
tw